In [1]:

!gdown --id 1m8BcFqCadenPM65rxNqW8L3drhoiTkSa  #before second training
!gdown --id 1jcrSlbN-M4GOYNIToJ7p3RY8rVHQpDfz  #tokeniser

!gdown --id 1gWfM4QpcUAGK5caq0EWgmqbxLAmsgB8Y  #after second training
!gdown --id 1HDELUG8g2vs0HDSnJqxsqAZJlsGQpkLu  #tokeniser



/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1m8BcFqCadenPM65rxNqW8L3drhoiTkSa
From (redirected): https://drive.google.com/uc?id=1m8BcFqCadenPM65rxNqW8L3drhoiTkSa&confirm=t&uuid=5c4b0e16-426d-4560-ab51-17bce18ff584
To: /content/indicbart_distil.pkl
100% 976M/976M [00:14<00:00, 65.6MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1jcrSlbN-M4GOYNIToJ7p3RY8rVHQpDfz
To: /content/indicbart_distil_tokenizer.pkl
100% 2.99M/2.99M [00:00<00:00, 24.9MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Opt

In [33]:
#test files

!gdown --id 1AJw8Nw8L9nQjz5oD2NUu8OtEn3ikFHzT


/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1AJw8Nw8L9nQjz5oD2NUu8OtEn3ikFHzT
To: /content/test.csv
100% 18.2M/18.2M [00:00<00:00, 64.2MB/s]


In [10]:
import pickle
import torch
import pandas as pd


with open('/content/indicbart_distil.pkl', 'rb') as f:
    model = pickle.load(f)

with open('/content/indicbart_distil_tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(64014, 1024, padding_idx=0)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(64014, 1024, padding_idx=0)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-5): 6 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): La

In [11]:
def file_read(path):
    return pd.read_csv(path)

In [34]:
import pandas as pd

df_test = file_read('/content/test.csv')
# df_test = file_read('/kaggle/input/data-testing/test.csv')

In [13]:
!pip install rouge_score
!pip install rouge


In [30]:
from rouge import Rouge
def predict_summary(text):
    inputs = tokenizer(text, return_tensors='pt', max_length=600, truncation=True)
    inputs.to("cuda")
    summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=40, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

def score(df):

    df['predicted_summary'] = df['Article'].apply(predict_summary)

    rouge = Rouge()


    def calculate_rouge(row):
        return rouge.get_scores(row['predicted_summary'], row['Summary'], avg=True)

    df['rouge'] = df.apply(calculate_rouge, axis=1)
    df['rouge-1'] = df['rouge'].apply(lambda x: x['rouge-1']['f'])
    df['rouge-2'] = df['rouge'].apply(lambda x: x['rouge-2']['f'])
    df['rouge-l'] = df['rouge'].apply(lambda x: x['rouge-l']['f'])

    print(df['rouge-1'].mean())
    print(df['rouge-2'].mean())

In [37]:
df_test = df_test[:1000]
score(df_test)

0.5042776129701365
0.39179096473798225


In [36]:
print("Rouge-1 score :" ,df_test['rouge-1'].mean())
print("Rouge-2 score :", df_test['rouge-2'].mean())
print("Rouge-L score :", df_test['rouge-l'].mean())

Rouge-1 score : 0.5042776129701365
Rouge-2 score : 0.39179096473798225
Rouge-L score : 0.47032883071533255
